In [6]:
import wikipedia
import wikipediaapi

Art and culture
Geography and places
Health and fitness
History and events
Mathematics and abstractions
Natural sciences and nature
People and self
Philosophy and thinking
Religion and spirituality
Social sciences and society
Technology and applied sciences

In [7]:
drink_list = wikipedia.search("drink", results=1000)
food_list = wikipedia.search("food", results=1000)
ancient_list = wikipedia.search("ancient", results=1000)
transportation_list = wikipedia.search("transportation", results=1000)
shows_list = wikipedia.search("shows", results=1000)
music_list = wikipedia.search("music", results=1000)
actors_list = wikipedia.search("actors", results=1000)
movies_list = wikipedia.search("movies", results=1000)
finance_list = wikipedia.search("finance", results=1000)
##
language_list = wikipedia.search("language", results=1000)
nature_list = wikipedia.search("nature", results=1000)
environment_list = wikipedia.search("environment", results=1000)
history_list = wikipedia.search("history", results=1000)
econ_list = wikipedia.search("economics", results=1000)
nature_list = wikipedia.search("nature", results=1000)
lit_list = wikipedia.search("literature", results=1000)
psych_list = wikipedia.search("psychology", results=1000)
behavioral = wikipedia.search("behavioral", results=1000)
global_list = wikipedia.search("global", results=1000)
cities_list = wikipedia.search("cities", results=1000)
anthropology = wikipedia.search("anthropology", results=1000)
##
art_culture = wikipedia.search("art culture", results=1000)
geo_places = wikipedia.search("geography places", results=1000)
health_fit = wikipedia.search("health fitness", results=1000)
history_events = wikipedia.search("history events", results=1000)
math = wikipedia.search("mathematics abstractions", results=1000)
natural_science = wikipedia.search("natural sciences nature", results=1000)
people_self = wikipedia.search("people self", results=1000)
philosophy = wikipedia.search("philosophy thinking", results=1000)
religion = wikipedia.search("religion spirituality", results=1000)
social_society = wikipedia.search("social sciences society", results=1000)
tech = wikipedia.search("technology applied sciences", results=1000)

In [8]:
all_articles = (art_culture + geo_places + health_fit + history_events + 
                math + natural_science + people_self + philosophy + religion
                + social_society + tech + anthropology + cities_list + global_list
                + behavioral + psych_list + lit_list + nature_list + econ_list 
                + history_list + environment_list + nature_list + language_list
                + finance_list + movies_list + actors_list + music_list + shows_list
                + transportation_list + ancient_list + food_list + drink_list)

In [9]:
all_articles_unique = list(set(all_articles))

In [10]:
import pickle
with open('unique_articles.pkl', 'wb') as f:
    pickle.dump(all_articles_unique, f)

In [21]:
def collect_articles():
    docs2_long_text = []
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI) 
    for i in range(len(all_articles_unique)):
        temp = wiki_wiki.page(all_articles_unique[i])
        if len(temp.text) > 300:
            docs2_long_text.append(temp.text)
        else: 
            continue
    return docs2_long_text

In [22]:
all_collected_articles = collect_articles()

In [23]:
import pickle
with open('collected_articles.pkl', 'wb') as f:
    pickle.dump(all_collected_articles, f)

In [24]:
import pandas as pd
df_corpus2 = pd.DataFrame()

In [25]:
df_corpus2['content'] = pd.Series(all_collected_articles)

In [28]:
! pip install textstat

  Using cached https://files.pythonhosted.org/packages/66/73/97bb64c89d6f2b24be6ad76007823e19b4c32ed4d484420b3ec6892ac440/textstat-0.5.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/30/6cc0c95f0b59ad4b3b9163bff7cdcf793cc96fac64cf398ff26271f5cf5e/repoze.lru-0.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/15/82/08a3629dce8d1f3d91db843bb36d4d7db6b6269d5067259613a0d5c8a9db/Pyphen-0.9.5-py2.py3-none-any.whl


In [29]:
import textstat

In [30]:
complexity = [textstat.flesch_kincaid_grade(doc) for doc in all_collected_articles]

In [31]:
df_corpus2['score'] = pd.Series(complexity)

In [33]:
with open('df_corpus2.pkl', 'wb') as f:
    pickle.dump(df_corpus2, f)

## Load Files and Vectorize 

In [39]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [41]:
with open(r"df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [42]:
vec = TfidfVectorizer(stop_words='english', max_features=2000)

In [43]:
vec = vec.fit(df_corpus2['content'])

In [45]:
#vec.vocabulary_

In [46]:
corpus2_vectors = vec.transform(df_corpus2['content']).toarray()

In [47]:
user_doc = "A watermelon is a type of edible fruit, they are 92% water. About 6% of a watermelon is sugar, which makes it very sweet. There are many different types of watermelon. Some have a green rind on the outside and a red-pink flesh on the inside, with brown seeds. Some can have yellow flesh, and some can be seedless. The green rind on the outside is not usually eaten, though it can be used as a vegetable. It can also be stewed or pickled. Most watermelons are oblong or spherical. In Japan, watermelons are grown in different shapes. Many people like to eat watermelon in the summer because the fruit is cool and refreshing. Watermelons are a great source of vitamin A, vitamin C, vitamin B6 and vitamin B1. They also contain potassium, magnesium, carotenoid antioxidant, and lycopene. Watermelons are fruits that come from a vine-like plant."

In [48]:
user_doc_vector = vec.transform([user_doc]).toarray()

In [49]:
user_doc_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [50]:
distances = cdist(user_doc_vector,
                  corpus2_vectors,
                  metric='cosine')[0]

In [51]:
distances.shape

(14216,)

In [52]:
ranking = np.argsort(distances)

In [53]:
ranking.shape

(14216,)

In [54]:
top_ten = ranking[:10]
top_ten

array([ 9761,  4036, 13028,  8170,  2029,  7052, 12517,  7038,   233,
        5636])

In [55]:
distances[top_ten]

array([0.6431534 , 0.67301318, 0.67374923, 0.67489732, 0.68369456,
       0.70282575, 0.70369595, 0.70818457, 0.70850181, 0.72808566])

In [56]:
df_corpus2.loc[ranking, 'content']

9761     Grass jelly, or leaf jelly, is a jelly-like de...
4036     Nate Green is an American author, writer, mark...
13028    Brown sugar is a sucrose sugar product with a ...
8170     Squash (also known as cordial in British Engli...
2029     Icing, or frosting, is a sweet, often creamy g...
7052     The green economy is defined as economy that a...
12517    Tea is an aromatic beverage commonly prepared ...
7038     Traditional ginger beer is a naturally sweeten...
233      Sugar is the generic name for sweet-tasting, s...
5636     Green River is a bright green, lime-flavored s...
6248     Global Green USA is the U.S. arm of Green Cros...
11693    Cream soda (also known as creme soda) is a swe...
1714     Mojito (; Spanish: [moˈxito]) is a traditional...
5530     Chaas (gu:છાશ, hi:छाछ) is a dahi (yogurt)-base...
6242     Ginger ale is a carbonated soft drink flavoure...
7301     Bubble tea (also known as pearl milk tea, bubb...
7714     A "fizz" is a mixed drink variation on the old.

## Second Model 

In [70]:
import textstat
import operator

def give_harder_level(text):
    """
    Takes in text and returns an easier level read
    """
    ##find all the articles that are easier than this article 
    all_harder_text = []
    input_score = textstat.flesch_kincaid_grade(text)
    for i in range(len(df_corpus2['score'])):
        if df_corpus2['score'][i] > input_score:
            all_harder_text.append(df_corpus2['content'][i])
    
    vec = TfidfVectorizer(stop_words='english', max_features=2000)
    vec = vec.fit(all_harder_text)
    corpus2_vectors = vec.transform(all_harder_text).toarray()
    user_doc = text
    user_doc_vector = vec.transform([user_doc]).toarray()    
    distances = cdist(user_doc_vector,
                  corpus2_vectors,
                  metric='cosine')[0]
    ranking = np.argsort(distances)
    top_ten = ranking[:5]
    
    best_matches = []
    for i in top_ten:
        temp = df_corpus2['content'][i]
        best_match.append(temp)
        
    print(distances[top_ten])
    return (best_matches)

In [73]:
sample = "Statistics involves math."

In [74]:
give_harder_level(sample)

[0.37440306 0.52521238 0.61830424 0.6586793  0.70057984]


['The Animation Show of Shows is a traveling selection of the year\'s best animated short films, curated and presented by Acme Filmworks founder, Ron Diamond. It began in 1998 with the aim of showing the most original, funny, intelligent short animated films from all over the world and presenting them at the major animation studios in order to inspire their animators and directors. Since 2007, some of the films are being published on DVDs.\n\nFilms included in the Shows\n1st Annual Animation Show of Shows • 1999\nVillage of Idiots by Eugene Fedorenka and Rose Newlove \nMy Grandmother Ironed the King\'s Shirts by Torill Kove \n3 Misses by Paul Driessen \nWhen the Day Breaks by Wendy Tilby and Amanda Forbis\n\n2nd Annual Animation Show of Shows • 2000\nFather and Daughter by Michael Dudok de Wit \nLa Pista by Gianluigi Toccafondo\nRun of the Mill by Borge Ring\nCrime and Punishment by Piotr Dumała\n\n3rd Annual Animation Show of Shows • 2001\nRadio Umanak by Marie José van der Linden\nA 